# 张正友标定法 Calibrate the Camera with ZhangZhengyou Method.

In [1]:
import numpy as np
import cv2
#glob模块的主要方法就是glob,该方法返回所有匹配的文件路径列表（list）
#该方法需要一个参数用来指定匹配的路径字符串，其返回的文件名只包括当前目录里的文件名，不包括子文件夹里的文件。
import glob
import os

In [2]:
def calib(inter_corner_shape, size_per_grid, img_dir,img_type):
    # criteria: only for subpix calibration, which is not used here.
    # criteria表示迭代次数，设置终止条件，迭代30次，或者移动0.01
    # criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    w,h = inter_corner_shape #输入棋盘的交点，11*8
    # cp_int: corner point in int form, save the coordinate of corner points in world sapce in 'int' form
    # 世界坐标系上交点的坐标，设Z轴在棋盘平面上，z=0
    cp_int = np.zeros((w*h,3), np.float32)
    cp_int[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)  #like (0,0,0), (1,0,0), (2,0,0) ....,(10,7,0).
    # cp_world: corner point in world space, save the coordinate of corner points in world space.
    cp_world = cp_int*size_per_grid   # 每格的尺寸
    
    obj_points = [] # the points in world space 世界坐标系
    img_points = [] # the points in image space (relevant to obj_points) 图像坐标系
    images = glob.glob(img_dir + os.sep + '**.' + img_type)
    for fname in images:
        img = cv2.imread(fname)
        gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # find the corners, cp_img: corner points in pixel space. 像素坐标系的坐标
        ret, cp_img = cv2.findChessboardCornersSB(gray_img, (w,h), None)
        print((fname),ret)
        # if ret is True, save.
        if ret == True:
            # 亚像素单位的像素坐标点，更精确一点
            # cv2.cornerSubPix(gray_img,cp_img,(11,11),(-1,-1),criteria)
            obj_points.append(cp_world)
            img_points.append(cp_img)
            # view the corners
            cv2.drawChessboardCorners(img, (w,h), cp_img, ret)
            cv2.imshow('FoundCorners',img)
            
            img_name=fname.split('\\')[-1]
            cv2.imwrite(".\\pic\\chess_result\\"+img_name,img) # 保存图片
            cv2.waitKey(1)
    cv2.destroyAllWindows()
    # calibrate the camera 相机匹配函数
    ret, mat_inter, coff_dis, v_rot, v_trans = cv2.calibrateCamera(obj_points, img_points, gray_img.shape[::-1], None, None)
    print (("ret:"),ret) #标定成功
    print (("internal matrix:\n"),mat_inter) # 内参数
    # in the form of (k_1,k_2,p_1,p_2,k_3)
    print (("distortion cofficients:\n"),coff_dis)  # 畸变系数
    print (("rotation vectors:\n"),v_rot)  #旋转向量
    print (("translation vectors:\n"),v_trans) #平移向量
    # calculate the error of reproject 投影误差
    total_error = 0
    for i in range(len(obj_points)):
        img_points_repro, _ = cv2.projectPoints(obj_points[i], v_rot[i], v_trans[i], mat_inter, coff_dis)
        error = cv2.norm(img_points[i], img_points_repro, cv2.NORM_L2)/len(img_points_repro)
        total_error += error
    print(("Average Error of Reproject: "), total_error/len(obj_points))
    
    
    # 畸变矫正
#     images = cv2.imread(".\\pic\\100011.png")
#     h,w=images.shape[:2]
#     newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mat_inter,coff_dis,(w,h),1,(w,h)) # 自由比例参数
#     dst = cv2.undistort(images, mat_inter, coff_dis, None, newcameramtx)
#     print(roi)
#     # clip the image
#     x,y,w,h = roi
#     dst = dst[y:y+h, x:x+w]
#     cv2.imwrite(".\\pic\\fixed.png", dst)
    
    return mat_inter, coff_dis
    



In [3]:
inter_corner_shape = (10,8) ##11*8
size_per_grid = 0.01
img_dir = ".\\pic\\chess1"  #RGB_camera_calib_img
img_type = "jpg"  #png
calib(inter_corner_shape, size_per_grid, img_dir,img_type)

.\pic\chess1\00001.jpg True
.\pic\chess1\00002.jpg True
.\pic\chess1\00003.jpg True
.\pic\chess1\00004.jpg True
.\pic\chess1\00005.jpg True
.\pic\chess1\00006.jpg True
.\pic\chess1\00007.jpg True
.\pic\chess1\00008.jpg True
.\pic\chess1\00009.jpg True
.\pic\chess1\00010.jpg True
.\pic\chess1\00011.jpg True
.\pic\chess1\00015.jpg True
ret: 0.6130781449993888
internal matrix:
 [[1.32379626e+03 0.00000000e+00 8.75728196e+02]
 [0.00000000e+00 1.32903996e+03 6.60034696e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion cofficients:
 [[ 0.11605086 -0.24534507  0.00616821  0.00613172 -0.39116163]]
rotation vectors:
 [array([[-0.10037094],
       [ 0.39136447],
       [-3.07475719]]), array([[-1.01005218],
       [ 0.40927754],
       [-2.83635486]]), array([[0.6644561 ],
       [0.36244988],
       [0.04584512]]), array([[-0.91874689],
       [-0.61016983],
       [ 2.78212087]]), array([[-0.83096057],
       [-0.91989745],
       [ 2.56704815]]), array([[ 1.07334485],
       [-

(array([[1.32379626e+03, 0.00000000e+00, 8.75728196e+02],
        [0.00000000e+00, 1.32903996e+03, 6.60034696e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[ 0.11605086, -0.24534507,  0.00616821,  0.00613172, -0.39116163]]))